In [1]:
pip install pyspark pymongo findspark matplotlib

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import to_timestamp, to_date, col
import findspark
from pymongo import MongoClient
import matplotlib.pyplot as plt

findspark.init()

# spark_session = SparkSession \
#     .builder \
#     .appName("myApp") \
#     .config("spark.mongodb.read.connection.uri", "mongodb://127.0.0.1/UserDB.Arrests") \
#     .config("spark.mongodb.write.connection.uri", "mongodb://127.0.0.1/UserDB.Arrests") \
#     .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.0') \
#     .getOrCreate()

spark_session = SparkSession \
    .builder \
    .appName("myApp") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/UserDB.Arrests") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/UserDB.Arrests") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
    .getOrCreate()

22/04/27 17:37:13 WARN Utils: Your hostname, Miros-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.9.161.107 instead (on interface en0)
22/04/27 17:37:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/miro/.ivy2/cache
The jars for the packages stored in: /Users/miro/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-17dfe8aa-9096-4554-9b21-79e9aac8c483;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central


:: loading settings :: url = jar:file:/opt/homebrew/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 93ms :: artifacts dl 4ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   4   |   0   |   0   |   0   ||   4   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-17dfe8aa-9096-4554-9b21-79e9aac8c483
	confs: [default]
	0 artifacts copied, 4 already retrieved (0kB/3ms)
22/04/27 17:37:13 WARN 

In [5]:
df: DataFrame = spark_session.read.format("mongo").load() # .withColumn("arrest_date", col("arrest_date").cast("string")) #.withColumn("arrest_date", to_date("arrest_date"))

In [6]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- address_number: string (nullable = true)
 |-- arrest_date: timestamp (nullable = true)
 |-- case_number: string (nullable = true)
 |-- charges: string (nullable = true)
 |-- city: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- hamilton_county_census_tracts_2010: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- longitude: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- zip: string (nullable = true)



In [7]:
df.show()

+--------------------+--------------+-------------------+-----------+--------------------+-----------+----------------+----------------------------------+------------------+--------------------+------------------+-----+------+-------------------+-----+
|                 _id|address_number|        arrest_date|case_number|             charges|       city|council_district|hamilton_county_census_tracts_2010|          latitude|            location|         longitude| race|   sex|        street_name|  zip|
+--------------------+--------------+-------------------+-----------+--------------------+-----------+----------------+----------------------------------+------------------+--------------------+------------------+-----+------+-------------------+-----+
|{624de5bda6fb4b39...|          7200|2022-04-04 19:00:00|  22-034544|     Serving Warrant|Chattanooga|               4|                            011446|         35.041401|{Point, [-85.1552...|        -85.155236|White|  Male|     Shallowfor

In [9]:
#converts DataFrame to rdd
rdd=df.rdd
print(rdd.take(2))

[Row(_id=Row(oid='624de5bda6fb4b3918b7bb01'), address_number='7200', arrest_date=datetime.datetime(2022, 4, 4, 19, 0), case_number='22-034544', charges='Serving Warrant', city='Chattanooga', council_district='4', hamilton_county_census_tracts_2010='011446', latitude='35.041401', location=Row(type='Point', coordinates=[-85.155236, 35.041401]), longitude='-85.155236', race='White', sex='Male', street_name='Shallowford Rd', zip='37421'), Row(_id=Row(oid='624de5bda6fb4b3918b7bb02'), address_number='2300', arrest_date=datetime.datetime(2022, 4, 4, 18, 24), case_number='22-034525', charges='Serving Warrant, Serving Warrant, Serving Warrant', city='Chattanooga', council_district='7', hamilton_county_census_tracts_2010='002400', latitude='35.003068999999996', location=Row(type='Point', coordinates=[-85.285782, 35.003069]), longitude='-85.28578200000001', race='Black', sex='Male', street_name='E 35th St', zip='37407')]


In [ ]:
rdd.getNumPartitions()

In [10]:
rdd4 = rdd.repartition(4)

In [ ]:
rdd4.getNumPartitions()

In [12]:
race_pairs = rdd4.map(lambda x: (x.race, 1))
race_count = race_pairs.reduceByKey(lambda x, y: x + y).collect()
print(race_count)
# plt.hist(raceCount)

[('White', 36054), ('Black', 36088), ('Other', 430)]


In [20]:
street_pairs = rdd4.map(lambda x: (x.street_name, 1))
street_count = street_pairs.reduceByKey(lambda x, y: x + y).takeOrdered(10, lambda s: -1 * s[1])
print(street_count)

[('Gunbarrel Rd', 3144), ('Lee Hwy', 2670), ('Brainerd Rd', 1752), ('Shallowford Rd', 1666), ('Greenway View Dr', 1644), ('Rossville Blvd', 1610), ('Market St', 1598), ('E 11th St', 1282), ('Hixson Pike', 1242), ('Cummings Hwy', 1240)]


In [21]:
gender_pairs = rdd4.map(lambda x: (x.sex, 1))
gender_count = gender_pairs.reduceByKey(lambda x, y: x + y).takeOrdered(10, lambda s: -1 * s[1])
print(gender_count)

[('Male', 51092), ('Female', 21478), (None, 2)]


In [ ]:
namesGrouped = race_pairs.groupByKey()
for key, value in namesGrouped.collect():
    print('{0}: {1}'.format(key, list(value)))

In [ ]:
from operator import add
nameCountsGrouped = namesGrouped.map(lambda x : (x[0], sum(x[1]))) 
print(nameCountsGrouped.collect())

In [ ]:
df.printSchema()